In [6]:
from pathlib import Path

import automech as am
import polars as pl
from project_utilities import p_

open_browser = False
tag = "Z_mess_v0"
root_path = Path("../..")

stoich = "C5H9O"
reactants = ["C5H8(522)", "OH(4)"]

calc_mech_json = p_.calculated_pes_mechanism(
    tag, stoich, "json", path=p_.data(root_path)
)
calc_mech = am.io.read(calc_mech_json)

if open_browser:
    disp_mech = calc_mech.model_copy(deep=True)
    disp_mech.reactions = disp_mech.reactions.filter(~pl.col("well_skipping"))
    am.display(disp_mech)

In [7]:
import functools
import itertools
import operator
from collections.abc import Sequence

import altair as alt
import autochem as ac
import numpy as np
import pint
import polars as pl
from automech import Mechanism, combine_enantiomers, consumption_mechanism, reaction
from automech.reaction import Reaction, ReactionRate, ReactionRateExtra
from automech.util import c_
from IPython.display import display as ipy_display


def display_reactant_branching_fractions(
    mech: Mechanism,
    reactants: Sequence[str],
    *,
    T_range: tuple[float, float] = (500, 1250),
    P_range: tuple[float, float] = (0.1, 100),
    T: float = 825,
    P: float = 1,
    T_drop: Sequence[float] = (725, 750, 775, 850, 875),
    extra_rates: dict[str, ac.rate.RateFit] | None = None,
    min_branch_frac: float | None = None,
):
    """Display reactant branching fractions

    :param mech: Mechanism
    :param reactants: Reactants
    :param T_range: Temperature range
    :param P_range: Pressure range
    :param T: Temperature
    :param P: Pressure
    """
    rev_mapping: dict[str, str] = {
        ReactionRate.rate: ReactionRateExtra.rev_rate,
        ReactionRateExtra.rate_data: ReactionRateExtra.rev_rate_data,
        ReactionRateExtra.branch_frac: ReactionRateExtra.rev_branch_frac,
    }  # type: ignore
    rate_cols = [*rev_mapping.keys(), *rev_mapping.values()]

    mech = consumption_mechanism(mech, reactants=reactants, rev_mapping=rev_mapping)
    mech = combine_enantiomers(mech, rate_cols=rate_cols)

    obj_pre = c_.temp()
    obj_mapping = c_.to_(rate_cols, obj_pre)
    rxn_df = reaction.with_rate_object_columns(mech.reactions, obj_mapping)

    # Column names
    well_skipping: str = ReactionRateExtra.well_skipping  # type: ignore

    # Sort by median branching fraction
    branch_frac: str = ReactionRateExtra.branch_frac  # type: ignore
    median_branch_frac = c_.temp()
    rxn_df = rxn_df.with_columns(
        pl.col(obj_mapping[branch_frac])
        .map_elements(lambda o: np.nanmedian(o.rate.k_data), return_dtype=pl.Float64)
        .alias(median_branch_frac)
    )
    rxn_df = rxn_df.sort(pl.col(median_branch_frac), descending=True)
    if min_branch_frac is not None:
        rxn_df = rxn_df.filter(pl.col(median_branch_frac) > min_branch_frac)

    # Add colors
    nrates = rxn_df.height
    color = c_.temp()
    colors = list(
        itertools.islice(itertools.cycle(ac.util.plot.LINE_COLOR_CYCLE), nrates)
    )
    rxn_df = rxn_df.with_columns(pl.Series(color, colors, dtype=pl.String))

    # Add labels
    products: str = Reaction.products  # type: ignore
    label = c_.temp()
    rxn_df = rxn_df.with_columns(pl.col(products).list.join("+").alias(label))

    # Branching fractions
    branch_frac_obj = obj_mapping[branch_frac]
    fs = [r.rate for r in rxn_df.get_column(branch_frac_obj).to_list()]
    f_labels = rxn_df.get_column(label).to_list()
    f_colors = rxn_df.get_column(color).to_list()
    if True:
        fs.insert(0, functools.reduce(operator.add, fs))
        f_labels.insert(0, "total")
        f_colors.insert(0, ac.util.plot.Color.black)

    f_chart = ac.rate.data.display_p(
        fs, T=T, P_range=P_range, color=f_colors, y_label="𝑓", y_unit=""
    )

    # Rates
    rate_data: str = ReactionRateExtra.rate_data  # type: ignore
    rate: str = ReactionRate.rate  # type: ignore
    rate_data_obj = obj_mapping[rate_data]
    rate_obj = obj_mapping[rate]
    k_datas = [r.rate for r in rxn_df.get_column(rate_data_obj).to_list()]
    k_datas = [r.drop_temperatures(T_drop) for r in k_datas]
    k_fits = [r.rate for r in rxn_df.get_column(rate_obj).to_list()]
    ks = [*k_datas, *k_fits]
    k_labels = rxn_df.get_column(label).to_list() * 2
    k_colors = rxn_df.get_column(color).to_list() * 2
    if True:
        ks.insert(0, functools.reduce(operator.add, k_datas))
        k_labels.insert(0, "Total")
        k_colors.insert(0, ac.util.plot.Color.black)

    if extra_rates is not None:
        nextra = len(extra_rates)
        extra_colors = itertools.islice(
            itertools.cycle(ac.util.plot.LINE_COLOR_CYCLE), nrates, nrates + nextra
        )
        for extra_label, extra_rate in extra_rates.items():
            ks.append(extra_rate)
            k_labels.append(extra_label)
            k_colors.append(next(extra_colors))

    k_chart = ac.rate.data.display(
        ks, T_range=T_range, P=P, label=k_labels, color=k_colors
    )

    chart = alt.vconcat(f_chart, k_chart).configure_axis(grid=False)
    # Rates
    ipy_display(chart)

    # # Labels
    # labels = list(map("+".join, rxn_df.get_column(products).to_list()))

    # fracs = [r.rate for r in rxn_df.get_column(branch_frac).to_list()]
    # fracs.insert(0, functools.reduce(operator.add, fracs))

    # from IPython.display import display

    # display(rxn_df)


kB = pint.Quantity("boltzmann_constant").m_as("J/K")
expt_rate = ac.rate.ArrheniusRateFit(
    order=2,
    A=1.64e-10,
    b=0,
    E=1660 * kB,
    T_min=856,
    T_max=1258,
    units={"substance": "molec", "energy": "J"},
)
reactants = ("C5H8(522)", "OH(4)")
display_reactant_branching_fractions(
    calc_mech, reactants, min_branch_frac=0.01, extra_rates={"expt": expt_rate}
)

alt.VConcatChart(...)

In [8]:
import urllib.parse

import altair as alt
import pandas as pd


# --- helper to convert SVG string to data URL ---
def svg_to_data_url(svg_str):
    return "data:image/svg+xml;utf8," + urllib.parse.quote(svg_str)


# --- SVG icons for legend ---
svgs = {
    "A": '<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">'
    '<circle cx="12" cy="12" r="10" fill="#1f77b4"/></svg>',
    "B": '<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">'
    '<rect x="2" y="2" width="20" height="20" fill="#ff7f0e"/></svg>',
    "C": '<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">'
    '<path d="M12 2 L22 22 L2 22 Z" fill="#2ca02c"/></svg>',
}

# --- build DataFrame ---
legend_df = pd.DataFrame(
    [{"category": name, "url": svg_to_data_url(svg)} for name, svg in svgs.items()]
)

order = list(svgs.keys())

# --- SVG-only legend ---
legend = (
    alt.Chart(legend_df)
    .mark_image(width=24, height=24)
    .encode(y=alt.Y("category:N", sort=order, axis=None), x=alt.value(10), url="url:N")
    .properties(width=50, height=100)  # small width for SVG-only
)
legend

alt.Chart(...)

In [9]:
import altair as alt
import pandas as pd

df = pd.DataFrame(
    {
        "x": [1, 2, 3, 1.5, 2.5, 3.2],
        "y": [2, 1, 3, 2.5, 1.2, 2.8],
        "category": ["A", "B", "C", "A", "B", "C"],
    }
)

main_chart = (
    alt.Chart(df)
    .mark_point(size=120, filled=True)
    .encode(
        x="x:Q",
        y="y:Q",
        color=alt.Color("category:N", legend=None),  # hide default legend
    )
    .properties(width=400, height=300)
)
main_chart

alt.Chart(...)

In [10]:
final_chart = alt.hconcat(main_chart, legend).configure_view(stroke=None)
final_chart


alt.HConcatChart(...)